In [ ]:
!conda install --file requirements.txt --yes

In [ ]:
import mbuild as mb
from foyer import Forcefield

In [ ]:
water = mb.load('files/spce.mol2')
water.name = 'water'

In [ ]:
water_box = mb.fill_box(
    compound=water,     
    n_compounds=3000,
    density=1000,
)

In [ ]:
spce = Forcefield('files/spce.xml')
system = spce.apply(water_box.to_parmed(residues=['water']))

In [ ]:
system.save('system.gro', combine='all')
system.save('system.top', combine='all')

In [ ]:
!gmx grompp -f files/npt.mdp -c system.gro -p system.top -o npt.tpr

In [ ]:
!gmx mdrun -v -deffnm npt

In [ ]:
import mdtraj as md
import numpy as np

trj = md.load('npt.xtc', top='system.gro')

In [ ]:
rho = md.density(trj)

ref = np.loadtxt('ref_data/rho.txt')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()

ax.plot(trj.time, rho, 'b-', label='Data')
ax.plot(ref[:, 0], ref[:, 1], 'k-', label='Reference')

ax.set_xlabel(('Simulation time, ps'))
ax.set_ylabel(('System density, kg/m^3'))

ax.legend(loc=0)